In [9]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

import pandas as pd 

In [10]:
py.plotly.tools.set_credentials_file(username='savannahvi', api_key='tncaIB087gdSYQQgDQVZ')

In [19]:
comic_data = pd.read_csv('comic_characters.csv')

In [12]:
comic_data['eyes'] = comic_data['eyes'].fillna('unknown')

In [48]:
comic_data['color'] = [1 if x == 'marvel' else 0 for x in comic_data['world']]

In [14]:
def encode_dict(col):
    '''
    creates a dictionary to encode the columns
    '''
    options = list(set(col))
    dict_opts = {}
    for i in range(len(options)):
        dict_opts[options[i]] = i
    return dict_opts


def encode(col, dic):
    return [x.replace(x,str(dic[x])) for x in col]

In [50]:
comic_data['align_id'] = encode(comic_data['align'], encode_dict(comic_data['align']))
comic_data['sex_id'] = encode(comic_data['sex'], encode_dict(comic_data['sex']))
comic_data['gsm_id'] = encode(comic_data['gsm'], encode_dict(comic_data['gsm']))
comic_data['alive_id'] = encode(comic_data['alive'], encode_dict(comic_data['alive']))

In [61]:
data_comic = [
    go.Parcoords(
        line = dict(color = comic_data['color'],
                   colorscale = [[0,'#0000FF'],[1,'#FF0000']]),
        dimensions = list([
            dict(range = [1935, 2014],
                 constraintrange = [1935,2013],
                label = 'Year', values = comic_data['year']),
            dict(tickvals = [0,1,2,3],
                ticktext = ['Good Character', 'Neutral Character', 'Bad Character', 'Unknown'],
                label = 'Align', values = comic_data['align_id']),
            dict(tickvals = [0,1,2,3,4,5],
                 ticktext = ['Agender Chractres', 'Transgender Characters', 'Unknown', 'Male Character', 'Female Character', 'Genderfluid'],
                label = 'Sex', values = comic_data['sex_id']),
            dict(tickvals = [0,1,2,3,4,5,6],
                 ticktext = ['Transgender', 'Pansexual', 'Transvestite', 'Homosexual', 'Heterosexual', 'Bisexual', 'Genderfluid'],
                label = 'Gsm', values = comic_data['gsm_id']),
            dict(tickvals = [0, 1, 2],
                 ticktext = ['Living', 'Deceased', 'Unknown'],
                label = 'Alive', values = comic_data['alive_id']),
            dict(tickvals = [0, 1],
                 ticktext = ['DC', 'Marvel'],
                label = 'World', values = comic_data['color'])
        ])
    )
]



fig = go.Figure(data = data_comic)
py.iplot(fig, filename = 'parcoords-basic')

# Bar Charts for counts each year

In [22]:
comic_data['count'] = 1
df_align = comic_data[['align', 'year', 'count']].groupby(['align', 'year']).count().reset_index()
df_align.columns = ['align', 'year', 'align_count']

df_sex = comic_data[['sex', 'year', 'count']].groupby(['sex', 'year']).count().reset_index()
df_sex.columns = ['sex', 'year', 'sex_count']

df_gsm = comic_data[['gsm', 'year', 'count']].groupby(['gsm', 'year']).count().reset_index()
df_gsm.columns = ['gsm', 'year', 'gsm_count']

df_alive = comic_data[['alive', 'year', 'count']].groupby(['alive', 'year']).count().reset_index()
df_alive.columns = ['alive', 'year', 'alive_count']

In [45]:


fig = tls.make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=('Alignment', 'Sex',
                                                          'GSM', 'Alive'))

for alignment in list(set(df_align['align'])):
    fig.append_trace({'x': df_align['year'], 'y': df_align[df_align['align'] == alignment]['align_count'], 'type': 'bar', 'name': 'align-{}'.format(alignment)}, 1, 1)
    
for sex in list(set(df_sex['sex'])):
    fig.append_trace({'x': df_sex['year'], 'y': df_sex[df_sex['sex'] == sex]['sex_count'], 'type': 'bar', 'name': 'sex-{}'.format(sex)}, 2, 1)

for gsm in list(set(df_gsm['gsm'])):
    fig.append_trace({'x': df_gsm['year'], 'y': df_gsm[df_gsm['gsm'] == gsm]['gsm_count'], 'type': 'bar', 'name': 'gsm-{}'.format(gsm)}, 3, 1)

for alive in list(set(df_alive['alive'])):
    fig.append_trace({'x': df_alive['year'], 'y': df_alive[df_alive['alive'] == alive]['alive_count'], 'type': 'bar', 'name': 'alive-{}'.format(alive)}, 4, 1)

    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



# trying to get a filter...

In [129]:
eye_color = {'Blue Eyes':'blue', 'Brown Eyes':'brown', 'Black Eyes':'black','Green Eyes':'green',
'Red Eyes':'red', 'White Eyes':'ivory', 'Yellow Eyes':'yellow','Grey Eyes':'grey',
'Hazel Eyes':'sienna', 'Variable Eyes':'beige', 'Photocellular Eyes':'lime', 'Purple Eyes':'purple',
'Orange Eyes':'orange', 'Pink Eyes':'pink', 'Gold Eyes':'gold','Violet Eyes':'violet', 'One Eye':'cyan',
'Amber Eyes':'godenrod', 'Silver Eyes':'silver', 'No Eyes':'darkblue', 'Multiple Eyes': 'indigo',
'Magenta Eyes':'magenta', 'Compound Eyes': 'beige', 'unknown':'white'}

In [137]:
opt = []
opts = []
for eye in list(set(comic_data['eyes'])):
    opt = dict(
        target = [eye], value = dict(marker = dict(color = eye_color[eye]))
    )
    opts.append(opt)

In [140]:
data_comic = [
    go.Parcoords(
        line = dict(color = comic_data['color'],
                   colorscale = [[0,'#0000FF'],[1,'#FF0000']]),
        dimensions = list([
            dict(range = [1935, 2014],
                 constraintrange = [1935,2013],
                label = 'Year', values = comic_data['year']),
            dict(tickvals = [0,1,2,3],
                ticktext = ['Good Character', 'Neutral Character', 'Bad Character', 'Unknown'],
                label = 'Align', values = comic_data['align_id']),
            dict(tickvals = [0,1,2,3,4,5],
                 ticktext = ['Agender Chractres', 'Transgender Characters', 'Unknown', 'Male Character', 'Female Character', 'Genderfluid'],
                label = 'Sex', values = comic_data['sex_id']),
            dict(tickvals = [0,1,2,3,4,5,6],
                 ticktext = ['Transgender', 'Pansexual', 'Transvestite', 'Homosexual', 'Heterosexual', 'Bisexual', 'Genderfluid'],
                label = 'Gsm', values = comic_data['gsm_id']),
            dict(tickvals = [0, 1, 2],
                 ticktext = ['Living', 'Deceased', 'Unknown'],
                label = 'Alive', values = comic_data['alive_id'])
        ])
    ),
    transforms = [
      dict(
        type = 'groupby',
        groups = comic_data['eyes'],
        styles = opts
      )] 
]

SyntaxError: invalid syntax (<ipython-input-140-0b50b3a850da>, line 23)